# Comprehensive Research: Federated Learning (FedAvg)

## 1. Concept Simulation
**Objective**: Train a model on distributed data without data ever leaving the client.
**Challenge**: Non-IID Data Distribution (Client A has different data than Client B).


In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# --- 1. FEDERATED DATA GENERATION ---
# Create Global Dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Split into Client Data Islands (Non-IID)
# Client A: Mostly Class 0
# Client B: Mostly Class 1
idx_0 = np.where(y == 0)[0]
idx_1 = np.where(y == 1)[0]

client_A_idx = np.concatenate([idx_0[:400], idx_1[:50]]) # 89% Class 0
client_B_idx = np.concatenate([idx_0[400:450], idx_1[50:450]]) # 89% Class 1

X_A, y_A = X[client_A_idx], y[client_A_idx]
X_B, y_B = X[client_B_idx], y[client_B_idx]

print(f"Client A: {len(y_A)} samples. Class 0 Ratio: {(y_A==0).mean():.2%}")
print(f"Client B: {len(y_B)} samples. Class 1 Ratio: {(y_B==1).mean():.2%}")

## 2. Infrastructure (Server & Clients)
Since Scikit-Learn doesn't support manual weight injection easily, we implement a micro Neural Network using Numpy to demonstrate the **Math of Averaging**.

In [ ]:
class SimpleNN:
    def __init__(self):
        # Weights: 20 inputs -> 1 output
        self.W = np.random.randn(20, 1) * 0.01
        self.b = np.zeros(1)
    
    def forward(self, X):
        return 1 / (1 + np.exp(-(X @ self.W + self.b)))
    
    def train(self, X, y, epochs=5, lr=0.1):
        for _ in range(epochs):
            # Simple SGD
            preds = self.forward(X).flatten()
            error = preds - y
            # Gradients
            dW = (X.T @ error.reshape(-1,1)) / len(X)
            db = np.mean(error)
            # Update
            self.W -= lr * dW
            self.b -= lr * db

    def get_weights(self):
        return self.W, self.b
    
    def set_weights(self, W, b):
        self.W = W
        self.b = b

# Initialize Global Model
global_model = SimpleNN()
acc_history = []

# --- 3. FEDERATED TRAINING LOOP ---
rounds = 20
for r in range(rounds):
    # 1. Server Broadcasts Weights
    W_global, b_global = global_model.get_weights()
    
    # 2. Clients Update Locally
    # Client A
    client_A = SimpleNN()
    client_A.set_weights(copy.deepcopy(W_global), copy.deepcopy(b_global))
    client_A.train(X_A, y_A, epochs=1) # Local Epochs
    W_A, b_A = client_A.get_weights()
    
    # Client B
    client_B = SimpleNN()
    client_B.set_weights(copy.deepcopy(W_global), copy.deepcopy(b_global))
    client_B.train(X_B, y_B, epochs=1)
    W_B, b_B = client_B.get_weights()
    
    # 3. Server Aggregates (FedAvg)
    # Weighted average based on sample size
    n_total = len(y_A) + len(y_B)
    W_new = (W_A * len(y_A) + W_B * len(y_B)) / n_total
    b_new = (b_A * len(y_A) + b_B * len(y_B)) / n_total
    
    global_model.set_weights(W_new, b_new)
    
    # 4. Evaluation (On balanced Global Test Set)
    preds = global_model.forward(X).flatten() > 0.5
    acc = (preds == y).mean()
    acc_history.append(acc)
    if r % 5 == 0:
        print(f"Round {r}: Global Accuracy = {acc:.2%}")

## 3. Convergence Analysis
Did the model learn? FedAvg usually learns slower than Centralized Training, but preserves privacy.

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(acc_history)
plt.title("Federated Learning Convergence")
plt.xlabel("Communication Round")
plt.ylabel("Global Accuracy")
plt.grid()
plt.show()